Leer las bases de datos que usaremos

In [ ]:
import pandas as pd

#c_p = pd.read_csv('Constructor_Performance.csv') #NOPE
#print(c_p.head()) # constructorResultsId  raceId  constructorId  points status

c_r = pd.read_csv('Constructor_Rankings.csv') #MAYBE
#c_r.columns() # constructorStandingsId	raceId	constructorId	points	position	positionText	wins

d_d = pd.read_csv('Driver_Details.csv') #MAYBE
#d_d.columns() # driverId	driverRef	number	code	forename	surname	dob	nationality	url

d_r = pd.read_csv('Driver_Rankings.csv') #YES <---
#d_r.columns() # driverStandingsId	raceId	driverId	points	position	positionText	wins

l_t = pd.read_csv('Lap_Timings.csv') #YES <---
#l_t.columns() # raceId	driverId	lap	position	time	milliseconds

#p_s = pd.read_csv('Pit_Stop_Records.csv') #NOPE
#p_s.columns() # raceId	driverId	stop	lap	time	duration	milliseconds

q_r = pd.read_csv('Qualifying_Results.csv') #MAYBE
#q_r.columns() # qualifyId	raceId	driverId	constructorId	number	position	q1	q2	q3

r_r = pd.read_csv('Race_Results.csv') #YES <---
#r_r.columns() # resultId	raceId	driverId	constructorId	number	grid	position	positionText	positionOrder	points	laps	time	milliseconds	fastestLap	rank	fastestLapTime	fastestLapSpeed	statusId

r_sc = pd.read_csv('Race_Schedule.csv') #YES
#r_sc.columns() # raceId	year	round	circuitId	name	date	time	url	fp1_date	fp1_time	fp2_date	fp2_time	fp3_date	fp3_time	quali_date	quali_time	sprint_date	sprint_time

r_st = pd.read_csv('Race_Status.csv') #MAYBE
#r_st.columns() # statusId	status

#s_s = pd.read_csv('Season_Summaries.csv') #NOPE
#s_s.columns() # year	url

#s_r = pd.read_csv('Sprint_Race_Results.csv') #NOPE
#print(s_r.head()) # 	resultId	raceId	driverId	constructorId	number	grid	position	positionText	positionOrder	points	laps	time	milliseconds	fastestLap	fastestLapTime	statusId

t_d = pd.read_csv('Team_Details.csv') #MAYBE
#t_d#.columns() # constructorId	constructorRef	name	nationality	url

#t_i = pd.read_csv('Track_Information.csv') #NOPE
#t_i.columns() # circuitId	circuitRef	name	location	country	lat	lng	alt	url

Obtención de los Id de Checo y MaxS

In [ ]:
checo_id = d_d[(d_d['forename']=='Sergio') & (d_d['surname']=='Pérez')]['driverId'].iloc[0]

max_id = d_d[(d_d['forename']=='Max') & (d_d['surname']=='Verstappen')]['driverId'].iloc[0]
max_id,checo_id

(np.int64(830), np.int64(815))

Definir los ID de carreras desde que Checo ingresó a Redbull {2021, 2022, 2023, 2024}

In [21]:
races_2021 = r_sc[(r_sc['year'] == 2021) & (r_sc['raceId'] <= 1131)]
id_races_2021 = races_2021.raceId
id_races_2021.shape[0] # Id de las carreras 2021
id_races_2021

1035    1053
1037    1052
1038    1051
1039    1054
1040    1055
1041    1056
1042    1057
1043    1058
1044    1059
1045    1060
1046    1061
1047    1062
1048    1063
1049    1064
1050    1065
1051    1066
1052    1067
1053    1069
1054    1070
1055    1071
1056    1072
1057    1073
Name: raceId, dtype: int64

In [28]:
races_2022 = r_sc[(r_sc['year'] == 2022) & (r_sc['raceId'] <= 1131)]
id_races_2022 = races_2022.raceId
id_races_2022.shape[0] # Id de las carreras 2022
id_races_2022

1036    1074
1058    1075
1059    1076
1060    1077
1061    1078
1062    1079
1063    1080
1064    1081
1065    1082
1066    1083
1067    1084
1068    1085
1069    1086
1070    1087
1071    1088
1072    1089
1073    1091
1074    1092
1075    1093
1076    1094
1077    1095
1078    1096
Name: raceId, dtype: int64

In [29]:
races_2023 = r_sc[(r_sc['year'] == 2023) & (r_sc['raceId'] <= 1131)]
id_races_2023 = races_2023.raceId
id_races_2023.shape[0] # Id de las carreras 2021
id_races_2023

1079    1098
1080    1099
1081    1100
1082    1101
1083    1102
1084    1104
1085    1105
1086    1106
1087    1107
1088    1108
1089    1109
1090    1110
1091    1111
1092    1112
1093    1113
1094    1114
1095    1115
1096    1116
1097    1117
1098    1118
1099    1119
1100    1120
Name: raceId, dtype: int64

In [30]:
races_2024 = r_sc[(r_sc['year'] == 2024) & (r_sc['raceId'] <= 1131)]
id_races_2024 = races_2024.raceId
print(id_races_2024.shape[0]) # Id de las carreras 2024
id_races_2024

11


1101    1121
1102    1122
1103    1123
1104    1124
1105    1125
1106    1126
1107    1127
1108    1128
1109    1129
1110    1130
1111    1131
Name: raceId, dtype: int64

Se efectuará un análisis por carrera con los timepo de vuelta de Checo y los de Max
Veremos si son normales o no
Verificar si los tiempos de vuelta de Max y Checo son considerablemente iguales con respectivas pruebas de hipótesis CHECO ID: 815      MAX ID: 830


In [23]:
from scipy.stats import kstest, norm
from scipy import stats
from scipy.stats import mannwhitneyu
import numpy as np

def test(a,b):
    alpha = .05
    stat, p_value = kstest(a, 'norm', args=(np.mean(a), np.std(a)))
    #print("Los datos SI siguen una distribución normal " if p_value > p else "Los datos NO siguen una distribución normal")
    #print(stat, p_value)
    n1 = True if p_value > alpha else False

    stat, p_value = kstest(b, 'norm', args=(np.mean(b), np.std(b)))
    #print("Los datos SI siguen una distribución normal " if p_value > p else "Los datos NO siguen una distribución normal")
    #print(stat, p_value)
    n2 = True if p_value > alpha else False
    #print(n1,n2, end='\t')

    state = None

    stat, p_value = mannwhitneyu(a, b, alternative='two-sided')

    #print(f'Estadístico U: {stat}')
    #print(f'Valor p: {p_value}')
    #print('Las muestras son significativamente diferentes' if p_value < alpha else 'No hay evidencia suficiente para decir que las muestras son diferentes', end='\t')
    state = True if p_value < alpha else False
    #print(state)
    return state#n1,n2,state


In [25]:
results_2021=[]
for id in id_races_2021:
    results_2021.append(test(l_t[(l_t['raceId']==id) & (l_t['driverId']==815)]['milliseconds'],l_t[(l_t['raceId']==id) & (l_t['driverId']==830)]['milliseconds']))

df_2021 = pd.DataFrame({
    'raceId': id_races_2021,
    'diff_performance': results_2021
})
df_2021

df_2021.to_latex('tabla1.tex', index=False) # Exportar dataframe a archivo .tex

/tmp/ipykernel_2769/3454742955.py:13: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  stat, p_value = kstest(b, 'norm', args=(np.mean(b), np.std(b)))
/tmp/ipykernel_2769/3454742955.py:21: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  stat, p_value = mannwhitneyu(a, b, alternative='two-sided')
/tmp/ipykernel_2769/3454742955.py:8: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  stat, p_value = kstest(a, 'norm', args=(np.mean(a), np.std(a)))
/home/rasv/Desktop/CIMA/Programación/F1_DataSet/.venv/lib/python3.12/site-packages/scipy/stats/_distn_infrastructure.py:2110: RuntimeWarning: invalid value encountered in divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)


In [31]:
results_2022=[]
for id in id_races_2022:
    results_2022.append(test(l_t[(l_t['raceId']==id) & (l_t['driverId']==815)]['milliseconds'],l_t[(l_t['raceId']==id) & (l_t['driverId']==830)]['milliseconds']))

df_2022 = pd.DataFrame({
    'raceId': id_races_2022,
    'diff_performance': results_2022
})
df_2022

df_2022.to_latex('tabla2.tex', index=False) # Exportar dataframe a archivo .tex

In [33]:
results_2023=[]
for id in id_races_2023:
    results_2023.append(test(l_t[(l_t['raceId']==id) & (l_t['driverId']==815)]['milliseconds'],l_t[(l_t['raceId']==id) & (l_t['driverId']==830)]['milliseconds']))

df_2023 = pd.DataFrame({
    'raceId': id_races_2023,
    'diff_performance': results_2023
})
df_2023
df_2023.to_latex('tabla3.tex', index=False) # Exportar dataframe a archivo .tex

/home/rasv/Desktop/CIMA/Programación/F1_DataSet/.venv/lib/python3.12/site-packages/scipy/stats/_distn_infrastructure.py:2110: RuntimeWarning: invalid value encountered in divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)


In [32]:
results_2024=[]
for id in id_races_2024:
    results_2024.append(test(l_t[(l_t['raceId']==id) & (l_t['driverId']==815)]['milliseconds'],l_t[(l_t['raceId']==id) & (l_t['driverId']==830)]['milliseconds']))

df_2024 = pd.DataFrame({
    'raceId': id_races_2024,
    'diff_performance': results_2024
})
df_2024
df_2024.to_latex('tabla4.tex', index=False) # Exportar dataframe a archivo .tex

/tmp/ipykernel_2769/3454742955.py:8: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  stat, p_value = kstest(a, 'norm', args=(np.mean(a), np.std(a)))
/tmp/ipykernel_2769/3454742955.py:21: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  stat, p_value = mannwhitneyu(a, b, alternative='two-sided')


ideas streamlit \\
1.Hacer gŕaficos por carrera con los tiempos de vuelta (verificar se pueden hacer varios trazos) \n
2. Hacer botones o "seleccionador" Con los año y/o carreras (por nombre de "GP") \n
3. webscraping de pilotos (quizá tambien pistas) con "seleccionador" usando bs4

ideas \n
1. establecer por mayoria, si hubo diferencia o no, ya sea total o por años
2. verificar si se puede exportar los datos a .tex
3. mostrar gŕaficas en tex?
